## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = list(range(1, 1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

22/01/02 19:19:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [8]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [9]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [10]:
# count
rdd.count()

1000

In [11]:
# first
rdd.first()

1

In [13]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [14]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [15]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [16]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([5.83721982e-01, 1.77438850e+00, 5.72050406e-01, 1.02885670e+00,
       2.23418290e+00, 4.95777054e+00, 4.57745271e+00, 7.57169345e+00,
       7.82602216e+00, 4.87814058e-01, 4.01395618e+00, 5.60822133e+00,
       5.85145299e+00, 1.72289254e+00, 6.16849486e+00, 1.10073645e+01,
       9.15913841e-01, 1.70948407e+01, 1.85438539e+01, 1.18915922e+01,
       3.95334703e+00, 1.80961361e+01, 7.52854950e+00, 2.34757945e+00,
       2.29425617e+01, 5.49794903e+00, 2.17681206e+01, 5.65891218e+00,
       1.24181870e+01, 2.16835157e+01, 2.43459910e+01, 1.21917371e+01,
       1.74421154e+01, 2.46160313e+01, 2.93598226e+01, 2.28005659e+01,
       2.04725252e+00, 2.96571819e+01, 3.62788944e+01, 2.62351448e+01,
       4.38449223e+00, 1.17901926e+01, 1.73642942e+01, 3.47927580e+01,
       1.22672992e+01, 3.95292822e+01, 2.31055970e+01, 1.51292101e+01,
       3.21932376e+01, 2.38368611e+01, 3.38995721e+01, 3.40154464e+01,
       2.69159199e+01, 8.39845175e-01, 3.34333511e+01, 2.75468066e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [18]:
price_items = sc.parallelize(sales_figures, 10)
price_items.take(5)

[0.5837219821912728,
 1.7743885027307795,
 0.572050406204094,
 1.0288566966607409,
 2.234182903036687]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [24]:
def sales_tax(num):
    return num*(1-0.08)

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [25]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.537024223615971,
 1.6324374225123173,
 0.5262863737077664,
 0.9465481609278816,
 2.055448270793752,
 4.561148898236007,
 4.211256491559475,
 6.965957972558404,
 7.199940389024209,
 0.4487889333076388]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [26]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [27]:
discounted.take(10)

[0.4833218012543739,
 1.4691936802610857,
 0.4736577363369898,
 0.8518933448350935,
 1.8499034437143769,
 4.105034008412406,
 3.7901308424035274,
 6.269362175302564,
 6.479946350121788,
 0.4039100399768749]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [30]:
price_items.map(sales_tax).map(lambda x: x*0.9).top(15)

[797.7417287832444,
 797.1648695128166,
 794.1392770987928,
 775.7847483254442,
 754.7028517211729,
 754.007477397115,
 753.0491606917482,
 750.6161836520231,
 748.7137695572495,
 746.9008948643619,
 744.9002680380822,
 741.129976884641,
 734.8736697182889,
 729.8059753969833,
 722.6946606223263]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [31]:
discounted.toDebugString()

b'(10) PythonRDD[15] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at readRDDFromFile at PythonRDD.scala:274 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [35]:
mapped = price_items.map(lambda x: (x, sales_tax(x)*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.5837219821912728, 0.4833218012543739), (1.7743885027307795, 1.4691936802610857), (0.572050406204094, 0.4736577363369898), (1.0288566966607409, 0.8518933448350935), (2.234182903036687, 1.8499034437143769), (4.957770541560876, 4.105034008412406), (4.57745270821682, 3.7901308424035274), (7.571693448433048, 6.269362175302564), (7.826022161982836, 6.479946350121788), (0.48781405794308563, 0.4039100399768749)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [36]:
flat_mapped = price_items.flatMap(lambda x: (x,sales_tax(x)*.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.5837219821912728, 0.4833218012543739, 1.7743885027307795, 1.4691936802610857, 0.572050406204094, 0.4736577363369898, 1.0288566966607409, 0.8518933448350935, 2.234182903036687, 1.8499034437143769]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [37]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)


# calculate total remaining in inventory 
selected_items.count()

285

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [39]:
selected_items.reduce(lambda x,y: x + y)

134248.59232919343

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [40]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(49, 311.8565163796642),
 (1, 302.75369536826116),
 (35, 323.9715194700649),
 (29, 300.59388673158986),
 (25, 315.40246288239643),
 (50, 357.0227486980097),
 (36, 300.8089652202046)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [43]:
# calculate how much each user spent
total = sales_data.reduceByKey(lambda x, y: x + y)
total.take(10)

[(10, 3095.5357247154006),
 (50, 784.1646799801654),
 (30, 3775.744252920757),
 (20, 1076.508969300958),
 (40, 3577.6780612714037),
 (1, 2459.2697226306236),
 (11, 5014.350724368714),
 (41, 2862.1791956455418),
 (21, 1970.9768689102293),
 (31, 2831.891446200382)]

In [44]:
# sort the users from highest to lowest spenders
total.sortBy(lambda x: x[1],ascending = False).collect()

[(28, 5530.122103198227),
 (11, 5014.350724368714),
 (37, 4673.924608775374),
 (13, 4518.588843367794),
 (14, 4512.607965045983),
 (18, 3800.7099593172557),
 (30, 3775.744252920757),
 (5, 3716.5911182175023),
 (3, 3596.2145721086918),
 (40, 3577.6780612714037),
 (35, 3444.4926085025068),
 (26, 3361.9432520414725),
 (6, 3351.9375376078115),
 (44, 3225.940562257567),
 (15, 3152.384992691271),
 (10, 3095.5357247154006),
 (42, 2968.3550291555885),
 (2, 2905.2828520733624),
 (25, 2901.457926963639),
 (12, 2882.6289809878253),
 (41, 2862.1791956455418),
 (49, 2833.7585156007244),
 (31, 2831.891446200382),
 (46, 2779.448783285236),
 (38, 2761.8119944713117),
 (22, 2703.0243745174375),
 (29, 2652.242726166921),
 (32, 2599.4272040772876),
 (43, 2576.21194048662),
 (33, 2475.7592570080315),
 (1, 2459.2697226306236),
 (23, 2358.993852150431),
 (36, 2263.7628030054666),
 (4, 2240.425254491105),
 (45, 2088.486761305761),
 (24, 2086.5625637909625),
 (8, 2018.711640107128),
 (16, 2000.0827817783538),

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [47]:
items_bought = sales_data.countByKey()
sorted(items_bought.items(),key=lambda kv:kv[1],reverse=True)

[(7, 12),
 (31, 11),
 (40, 9),
 (36, 9),
 (46, 9),
 (11, 8),
 (22, 8),
 (35, 7),
 (24, 7),
 (17, 7),
 (21, 7),
 (28, 7),
 (34, 7),
 (6, 7),
 (48, 7),
 (50, 7),
 (41, 7),
 (37, 6),
 (20, 6),
 (30, 6),
 (12, 6),
 (39, 6),
 (8, 6),
 (47, 6),
 (9, 6),
 (43, 5),
 (13, 5),
 (27, 5),
 (10, 5),
 (38, 5),
 (19, 5),
 (32, 5),
 (29, 5),
 (1, 5),
 (16, 5),
 (45, 5),
 (5, 4),
 (26, 4),
 (4, 4),
 (49, 4),
 (14, 4),
 (23, 4),
 (25, 4),
 (15, 3),
 (44, 3),
 (42, 3),
 (18, 3),
 (2, 2),
 (3, 2),
 (33, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
